In [61]:
# download json files

import urllib.request
import requests
import time

# import json
# source: https://api.tfl.gov.uk/Line/Route?serviceTypes=Regular
# get from website
url_routes = "https://api.tfl.gov.uk/Line/Route?serviceTypes=Regular"
file_routes = requests.get(url)
file_all_routes = open("Data/Route.json")
routes_json = json.load(file_all_routes)

count = 0
for r in routes_json:
    id = r["id"]
    name = r["name"]
    mode = r["modeName"]
    url = "https://api.tfl.gov.uk/Line/" + id + "/Route/Sequence/all?serviceTypes=Regular&excludeCrowding=true"
    # print(url)
    r = requests.get(url, allow_redirects=True)
    file_name = "Data/Routes/route_" + mode + "_" + id + ".json"
    open(file_name, "wb").write(r.content)

    print(id)
    count += 1
    if count == 40:
        time.sleep(40)
        count = 0

print("Done.")

154
155
156
157
158
159
16
160
161
162
163
164
165
166
167
169
17
170
171
172
173
174
175
176
177
178
179
18
180
181
182
183
184
185
186
187
188
189
19
190
191
192
193
194
195
196
197
198
199
2
20
200
201
202
203
204
205
206
207
208
209
21
210
211
212
213
214
215
216
217
218
219
22
220
221
222
224
225
226
227
228
229
23
230
231
232
233
234
235
236
237
238
24
240
241
242
243
244
245
246
247
248
249
25
250
251
252
253
254
255
256
257
258
259
26
260
261
262
263
264
265
266
267
268
269
27
270
272
273
274
275
276
277
278
279
28
280
281
282
283
284
285
286
287
288
289
29
290
291
292
293
294
295
296
297
298
299
3
30
300
301
302
303
304
306
307
308
309
31
312
313
314
315
316
317
318
319
32
320
321
322
323
324
325
326
327
328
329
33
330
331
333
335
336
337
34
340
341
343
344
345
346
347
349
35
350
352
353
354
355
356
357
358
359
36
360
362
363
364
365
366
367
368
37
370
371
372
375
376
377
378
379
38
380
381
382
383
384
385
386
388
389
39
390
393
394
395
396
397
398
399
4
40
401
403
404
405
406

In [7]:
# Converts Routes into GeoJSON file


import json
import requests

# import json
# source: https://api.tfl.gov.uk/Line/Route?serviceTypes=Regular
# get from website
# url_routes = "https://api.tfl.gov.uk/Line/Route?serviceTypes=Regular"
# file_routes = requests.get(url)
file_all_routes = open("Data/Route.json")
routes_json = json.load(file_all_routes)

#show data
# for d in routes_json:
#     print(d["modeName"],d["name"])

# output file
bus_routes = {
    "type": "FeatureCollection",
    "Features": []
}

tube_routes = {
    "type": "FeatureCollection",
    "Features": []
}

# add routes
count = 0
for r in routes_json:
    id = r["id"]
    name = r["name"]
    mode = r["modeName"]
    
    # get JSON for route
    file_name = "Data/Routes/route_" + mode + "_" + id + ".json"
    file_get = open(file_name)
    loaded_file = json.load(file_get)

    # print(loaded_file["lineStrings"][0])

    # store geometry as MultiLineString
    geometry = []
    for i in loaded_file["lineStrings"]:
        line = json.loads(i)
        for j in line:
            geometry.append(j)
    
    # print(geometry)

    # store in GeoJSON
    route = {
        "type": "feature",
        "geometry": {
            "type": "MultiLineString",
            "coordinates": geometry,
        },
        "properties": {
            "mode": mode,
            "id": id,
            "name": name,
        }
    }

    # output buses
    if mode == "bus":
        bus_routes["Features"].append(route)

    # output tube
    if mode == "tube":
        tube_routes["Features"].append(route)
    

    print(id)

routes_output = json.dumps(bus_routes)
with open("Data/routes_output.geojson", "w") as f:
    f.write(routes_output)

print("Done")



1
100
101
102
103
104
105
106
107
108
109
11
110
111
112
113
114
115
116
117
118
119
12
120
121
122
123
124
125
126
127
128
129
13
130
131
132
133
134
135
136
137
138
139
14
140
141
142
143
144
145
146
147
148
149
15
150
151
152
153
154
155
156
157
158
159
16
160
161
162
163
164
165
166
167
169
17
170
171
172
173
174
175
176
177
178
179
18
180
181
182
183
184
185
186
187
188
189
19
190
191
192
193
194
195
196
197
198
199
2
20
200
201
202
203
204
205
206
207
208
209
21
210
211
212
213
214
215
216
217
218
219
22
220
221
222
224
225
226
227
228
229
23
230
231
232
233
234
235
236
237
238
24
240
241
242
243
244
245
246
247
248
249
25
250
251
252
253
254
255
256
257
258
259
26
260
261
262
263
264
265
266
267
268
269
27
270
272
273
274
275
276
277
278
279
28
280
281
282
283
284
285
286
287
288
289
29
290
291
292
293
294
295
296
297
298
299
3
30
300
301
302
303
304
306
307
308
309
31
312
313
314
315
316
317
318
319
32
320
321
322
323
324
325
326
327
328
329
33
330
331
333
335
336
337
34
340
34